In [32]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [61]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
import pickle
import json
import h5py
import pandas as pd

def load_file(path, name):
    import importlib.util
    spec = importlib.util.spec_from_file_location(name, path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

models = load_file("../../utils/models.py", "models")


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_ref = pickle.load(open('../../data/infos_pca128.pytorch', 'rb'))
# on my system os error often pops up
# try to open the file explorer and copy paste the path again 
# https://drive.google.com/open?id=1pPf-7AmUVceVfgfmKEJ6ireEDKEJHw-7
f = h5py.File("../../data/static_ml20m.hdf5", "r")
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../../data/metadata.json'))

In [3]:
ddpg = models.Actor(1290, 128, 256).to(cuda)
td3 = models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../../models/DDPG/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../../models/TD3/td3_policy_1.pt'))

batch_size = 5000
n_batches = (f['done'].shape[0] // batch_size) - 1

In [4]:
test_batch = [f[key][n_batches*batch_size:(n_batches+1)*batch_size] for key in
                 ['movies', 'ratings', 'done']]


movies, ratings, done = test_batch
movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
state = torch.cat([movies_tensor[:, :-1, :].view(batch_size, -1),
                       ratings[:, :-1]], 1).to(cuda)


In [5]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}

In [6]:
def rank(gen_action, metric):
    scores = []
    for i in movie_ref.keys():
        scores.append([i, metric(movie_ref[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [7]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [8]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,58452,11.435287,Saving Sarah Cain,Drama Family,en,2007,5.5
1,217,11.806768,The Babysitter,Drama Thriller,en,1995,4.5
2,106436,12.002036,For Those in Peril,Drama,en,2013,6.4
3,105238,12.012250,New Town Killers,Thriller Drama,en,2008,5.7
4,4934,12.019874,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
5,36046,12.053860,Lightning Bug,Drama Thriller Horror,en,2004,6.1
6,32923,12.132998,Frankenfish,TV Movie Action Horror Comedy Sci-Fi Thriller,en,2004,4.7
7,108007,12.154035,Child Bride,Drama,en,1938,4.9
8,5340,12.176963,Joe,Drama Thriller,en,1970,6.2
9,7474,12.230637,The Boys in the Band,Drama,en,1970,7.0


In [9]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,58452,0.625712,Saving Sarah Cain,Drama Family,en,2007,5.5
1,217,0.681837,The Babysitter,Drama Thriller,en,1995,4.5
2,5994,0.682123,Nicholas Nickleby,Adventure Drama Action Family,en,2002,6.9
3,1564,0.682206,Roseanna's Grave,Comedy Romance Drama,en,1997,6.2
4,536,0.688057,A Simple Twist of Fate,Drama Family Comedy,en,1994,5.9
5,997,0.697582,Caught,Action Drama Thriller,en,1996,6.4
6,106436,0.704154,For Those in Peril,Drama,en,2013,6.4
7,72483,0.706626,Keith,Drama Family Romance,en,2008,7.2
8,32923,0.707664,Frankenfish,TV Movie Action Horror Comedy Sci-Fi Thriller,en,2004,4.7
9,91571,0.714338,Coriolanus,Drama Thriller,en,2011,6.0


In [10]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,58452,0.630589,Saving Sarah Cain,Drama Family,en,2007,5.5
1,1564,0.676273,Roseanna's Grave,Comedy Romance Drama,en,1997,6.2
2,5994,0.678979,Nicholas Nickleby,Adventure Drama Action Family,en,2002,6.9
3,217,0.679034,The Babysitter,Drama Thriller,en,1995,4.5
4,536,0.694473,A Simple Twist of Fate,Drama Family Comedy,en,1994,5.9
5,997,0.702358,Caught,Action Drama Thriller,en,1996,6.4
6,106436,0.702632,For Those in Peril,Drama,en,2013,6.4
7,3860,0.705442,The Opportunists,Comedy Crime Drama Romance,en,2000,5.3
8,108007,0.707519,Child Bride,Drama,en,1938,4.9
9,32923,0.710328,Frankenfish,TV Movie Action Horror Comedy Sci-Fi Thriller,en,2004,4.7


In [11]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,114758,77.330362,Open Windows,Thriller Action Crime,en,2014,5.1
1,58452,77.621997,Saving Sarah Cain,Drama Family,en,2007,5.5
2,33081,77.647231,Love Comes Softly,Drama Romance TV Movie Western,en,2003,6.4
3,70728,77.959979,Bronson,Drama Action Crime,en,2008,7.0
4,111653,78.009074,Nightwing,Horror,en,1979,5.3
5,3190,78.336959,Supernova,Horror Sci-Fi Thriller,en,2000,4.8
6,36289,78.532078,Asterix & Obelix Take on Caesar,Family Adventure Comedy Fantasy,fr,1999,6.0
7,3860,78.775890,The Opportunists,Comedy Crime Drama Romance,en,2000,5.3
8,7039,78.835438,Thunderheart,Action Adventure Crime Mystery Thriller Drama,en,1992,6.3
9,5994,78.839328,Nicholas Nickleby,Adventure Drama Action Family,en,2002,6.9


In [12]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,58452,11.435287,Saving Sarah Cain,Drama Family,en,2007,5.5
1,217,11.806768,The Babysitter,Drama Thriller,en,1995,4.5
2,106436,12.002036,For Those in Peril,Drama,en,2013,6.4
3,105238,12.012250,New Town Killers,Thriller Drama,en,2008,5.7
4,4934,12.019874,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
5,36046,12.053860,Lightning Bug,Drama Thriller Horror,en,2004,6.1
6,32923,12.132998,Frankenfish,TV Movie Action Horror Comedy Sci-Fi Thriller,en,2004,4.7
7,108007,12.154035,Child Bride,Drama,en,1938,4.9
8,5340,12.176963,Joe,Drama Thriller,en,1970,6.2
9,7474,12.230637,The Boys in the Band,Drama,en,1970,7.0


In [13]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,92676,2.367672,Thirteen Women,Drama Mystery Thriller,en,1932,6.3
1,95427,2.392205,The Silent Scream,Horror Thriller,en,1979,5.9
2,104101,2.481557,Bloody Murder,Horror Thriller,en,2000,3.3
3,58452,2.493490,Saving Sarah Cain,Drama Family,en,2007,5.5
4,70571,2.500651,Just Sex and Nothing Else,Drama Comedy Romance,hu,2005,6.8
5,69842,2.528371,Good Dick,Comedy Drama Romance,en,2008,6.2
6,6532,2.539029,Up the Sandbox,Drama Comedy,en,1972,6.9
7,78422,2.557258,How About You...,Comedy Drama,en,2007,6.0
8,130294,2.562311,The Tree,Drama,sl,2014,7.0
9,5676,2.579190,The Young Unknowns,Drama,en,2000,0.0


In [14]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,58452,0.663937,Saving Sarah Cain,Drama Family,en,2007,5.5
1,3860,0.702855,The Opportunists,Comedy Crime Drama Romance,en,2000,5.3
2,5994,0.704203,Nicholas Nickleby,Adventure Drama Action Family,en,2002,6.9
3,70728,0.710573,Bronson,Drama Action Crime,en,2008,7.0
4,36046,0.719018,Lightning Bug,Drama Thriller Horror,en,2004,6.1
5,997,0.727998,Caught,Action Drama Thriller,en,1996,6.4
6,5441,0.733042,Traces of Red,Drama Action Thriller Crime Mystery,en,1992,5.8
7,91571,0.734207,Coriolanus,Drama Thriller,en,2011,6.0
8,61361,0.734878,The Women,Family Comedy Drama Romance,en,2008,4.8
9,120621,0.735849,Savannah,Romance History Drama Family,en,2013,6.2


In [15]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,58452,102.036636,Saving Sarah Cain,Drama Family,en,2007,5.5
1,36046,107.603439,Lightning Bug,Drama Thriller Horror,en,2004,6.1
2,7474,108.486908,The Boys in the Band,Drama,en,1970,7.0
3,106436,108.922333,For Those in Peril,Drama,en,2013,6.4
4,108953,109.014084,Crave,Drama Thriller,en,2012,4.7
5,72153,109.988358,A Dangerous Woman,Drama Romance,en,1993,5.2
6,108007,110.055267,Child Bride,Drama,en,1938,4.9
7,5340,110.326385,Joe,Drama Thriller,en,1970,6.2
8,108092,110.354080,The Raven,Horror Thriller,en,2006,2.0
9,217,110.599770,The Babysitter,Drama Thriller,en,1995,4.5


# TD3

In [16]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [17]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,69487,8.969159,Anna Lucasta,Drama,en,1958,5.7
1,975,9.064531,Something to Sing About,Comedy,en,1937,5.4
2,3045,9.066180,Peter's Friends,Comedy Drama,en,1992,6.8
3,4934,9.077646,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
4,1170,9.081861,Best of the Best 3: No Turning Back,Action,en,1995,5.7
5,26147,9.155313,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2
6,72153,9.168726,A Dangerous Woman,Drama Romance,en,1993,5.2
7,1574,9.175961,Fall,Drama Romance,en,1997,7.0
8,6159,9.187111,All the Real Girls,Drama Romance,en,2003,6.1
9,88823,9.194849,High Time,Comedy,en,1960,5.5


In [18]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,77778,0.647918,Aces and Eights,Western Action,en,1936,6.0
1,86451,0.680345,Love Torn in a Dream,Drama Fantasy,fr,2000,8.0
2,3630,0.685550,Lisa and the Devil,Fantasy Horror Sci-Fi,it,1973,6.3
3,25736,0.688928,Dr. Jekyll and Mr. Hyde,Drama Horror Sci-Fi,en,1920,6.9
4,50208,0.692451,Rolling Family,Drama Comedy,en,2004,6.3
5,5852,0.694711,Saturday the 14th,Comedy Fantasy Horror,en,1981,4.9
6,59040,0.696896,Gabriel,Fantasy Horror Action Sci-Fi,en,2007,5.1
7,5895,0.699415,Making Contact,Sci-Fi Fantasy Horror Adventure,en,1985,5.9
8,72583,0.700828,Yeelen,Drama,bm,1987,6.8
9,7267,0.701087,Who's the Man?,Drama Action Comedy,en,1993,5.4


In [19]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,77778,0.652406,Aces and Eights,Western Action,en,1936,6.0
1,25736,0.683343,Dr. Jekyll and Mr. Hyde,Drama Horror Sci-Fi,en,1920,6.9
2,86451,0.687641,Love Torn in a Dream,Drama Fantasy,fr,2000,8.0
3,3630,0.689738,Lisa and the Devil,Fantasy Horror Sci-Fi,it,1973,6.3
4,4405,0.690891,The Last Laugh,Drama,de,1924,7.8
5,5895,0.693315,Making Contact,Sci-Fi Fantasy Horror Adventure,en,1985,5.9
6,5852,0.693351,Saturday the 14th,Comedy Fantasy Horror,en,1981,4.9
7,50208,0.697614,Rolling Family,Drama Comedy,en,2004,6.3
8,59040,0.699240,Gabriel,Fantasy Horror Action Sci-Fi,en,2007,5.1
9,7267,0.699413,Who's the Man?,Drama Action Comedy,en,1993,5.4


In [20]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,7023,77.250642,The Wedding Banquet,Comedy Drama Romance,zh,1993,7.2
1,218,78.227205,Boys on the Side,Comedy Drama,en,1995,6.4
2,1782,78.474345,Little City,Comedy Romance,en,1997,6.0
3,80160,79.430135,The Flintstone Kids' Just Say No Special,Animation,en,1988,5.6
4,130848,79.577274,Not My Type,Comedy Romance Drama,en,2014,6.2
5,95218,80.124119,First Position,Comedy Documentary Drama,en,2011,7.8
6,27095,80.190819,The Hole,Drama Fantasy Musical Romance,zh,1998,7.2
7,7125,80.224214,Spring Forward,Drama,en,1999,6.5
8,4400,80.231976,Donovan's Brain,Drama Horror Sci-Fi,en,1953,5.8
9,69906,80.394108,Possessed,Drama Horror,en,2000,4.5


In [21]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,69487,8.969159,Anna Lucasta,Drama,en,1958,5.7
1,975,9.064531,Something to Sing About,Comedy,en,1937,5.4
2,3045,9.066180,Peter's Friends,Comedy Drama,en,1992,6.8
3,4934,9.077646,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
4,1170,9.081861,Best of the Best 3: No Turning Back,Action,en,1995,5.7
5,26147,9.155313,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2
6,72153,9.168726,A Dangerous Woman,Drama Romance,en,1993,5.2
7,1574,9.175961,Fall,Drama Romance,en,1997,7.0
8,6159,9.187111,All the Real Girls,Drama Romance,en,2003,6.1
9,88823,9.194849,High Time,Comedy,en,1960,5.5


In [22]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,1021,1.870850,Angels in the Outfield,Comedy Drama,en,1994,5.9
1,8617,1.914344,BUtterfield 8,Drama,en,1960,6.7
2,115622,1.920735,Disaster L.A.: The Last Zombie Apocalypse Begi...,Action Horror Sci-Fi,en,2014,4.0
3,8915,2.012688,Stage Beauty,Comedy Romance,en,2004,7.0
4,81447,2.020318,"Smell of Camphor, Fragrance of Jasmine",Drama,fa,2000,5.0
5,1122,2.051540,Plutonium Circus,Documentary,en,1994,6.5
6,122938,2.057649,Melbourne,Drama,fa,2014,6.6
7,6038,2.057890,Tune in Tomorrow...,Comedy Drama Romance,en,1990,5.5
8,8892,2.058906,Basket Case 3,Horror,en,1991,4.8
9,5977,2.062714,Where’s Poppa?,Comedy,en,1970,6.8


In [23]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,7267,0.723282,Who's the Man?,Drama Action Comedy,en,1993,5.4
1,128848,0.726488,The Great Los Angeles Earthquake,Drama Action,en,1990,6.5
2,119629,0.727991,Exorcismus,Thriller Horror,en,2010,5.4
3,5659,0.732685,The Rocking Horse Winner,Drama,en,1949,6.3
4,25900,0.737640,The Curse of the Cat People,Drama Fantasy Horror,en,1944,6.2
5,80160,0.739324,The Flintstone Kids' Just Say No Special,Animation,en,1988,5.6
6,77778,0.739941,Aces and Eights,Western Action,en,1936,6.0
7,171,0.740888,Jeffrey,Comedy Drama Romance,en,1995,6.7
8,69906,0.744512,Possessed,Drama Horror,en,2000,4.5
9,84019,0.745253,Cool It,Documentary,en,2010,6.7


In [24]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,69487,78.976471,Anna Lucasta,Drama,en,1958,5.7
1,6159,79.734695,All the Real Girls,Drama Romance,en,2003,6.1
2,3045,80.318222,Peter's Friends,Comedy Drama,en,1992,6.8
3,1574,81.131599,Fall,Drama Romance,en,1997,7.0
4,88823,81.233994,High Time,Comedy,en,1960,5.5
5,1170,81.677628,Best of the Best 3: No Turning Back,Action,en,1995,5.7
6,26147,81.789108,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2
7,3187,81.850800,Trans,Drama,en,1998,5.0
8,93731,81.886749,A Warm December,Drama Romance,en,1973,6.4
9,70762,82.059685,The Curiosity of Chance,Comedy Drama,en,2006,6.0


# BCQ

In [62]:
bcq_actor = models.BCQ_Actor(1290, 128, 256).to(cuda)
bcq_vae = models.BCQ_VAE(1290, 128, 512).to(cuda)

bcq_actor.load_state_dict(torch.load('../../models/BCQ/bcq_policy.pt'))
bcq_vae.load_state_dict(torch.load('../../models/BCQ/bcq_vae.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [63]:
sampled_actions = bcq_vae.decode(state)
perturbed_actions= bcq_actor(state, sampled_actions)

In [64]:
bcq_action = perturbed_actions[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [65]:
rank(bcq_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,4186,9.822283,The Fortune Cookie,Romance Comedy,en,1966,7.4
1,4168,9.834513,Get Over It,Comedy Romance,en,2001,5.4
2,6917,9.870267,Promised Land,Drama Romance,en,1987,5.1
3,523,9.964449,Ruby in Paradise,Drama Romance,en,1993,6.2
4,360,9.976474,I Love Trouble,Action Comedy Romance,en,1994,5.4
5,2356,10.057261,Celebrity,Drama Comedy,en,1998,6.0
6,90576,10.131841,What's Your Number?,Comedy Romance,en,2011,6.4
7,78574,10.160169,Winter's Bone,Drama,en,2010,6.9
8,80789,10.198572,Loving Leah,Drama Romance TV Movie,en,2009,6.5
9,97705,10.206452,Where the Boys Are '84,Comedy Romance,en,1984,4.2


In [66]:
rank(bcq_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,360,0.600891,I Love Trouble,Action Comedy Romance,en,1994,5.4
1,100505,0.656930,Monte Walsh,Action Western,en,2003,5.8
2,102,0.659041,Mr. Wrong,Comedy Romance,en,1996,3.9
3,26649,0.660359,Lonesome Dove,Adventure Drama Western,en,1989,8.7
4,6311,0.668175,The Other Side of Heaven,Action Adventure Drama Family,en,2001,6.4
5,72142,0.670780,Love Exposure,Action Comedy Drama Romance,ja,2008,8.2
6,4890,0.677211,Shallow Hal,Comedy Romance,en,2001,5.9
7,5591,0.679854,Monkey Trouble,Comedy Family,en,1994,5.7
8,102497,0.686678,Last Stand at Saber River,TV Movie Western Action Drama,en,1997,5.6
9,7051,0.688065,What's New Pussycat?,Comedy Romance,en,1965,6.3


In [67]:
rank(bcq_action, distance.correlation)

,id,score,title,genres,language,year,rating
0,360,0.602163,I Love Trouble,Action Comedy Romance,en,1994,5.4
1,6311,0.658124,The Other Side of Heaven,Action Adventure Drama Family,en,2001,6.4
2,100505,0.658528,Monte Walsh,Action Western,en,2003,5.8
3,102,0.659895,Mr. Wrong,Comedy Romance,en,1996,3.9
4,26649,0.661879,Lonesome Dove,Adventure Drama Western,en,1989,8.7
5,5591,0.669769,Monkey Trouble,Comedy Family,en,1994,5.7
6,72142,0.672363,Love Exposure,Action Comedy Drama Romance,ja,2008,8.2
7,4890,0.675238,Shallow Hal,Comedy Romance,en,2001,5.9
8,102497,0.679603,Last Stand at Saber River,TV Movie Western Action Drama,en,1997,5.6
9,7051,0.681012,What's New Pussycat?,Comedy Romance,en,1965,6.3


In [68]:
rank(bcq_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,360,72.653805,I Love Trouble,Action Comedy Romance,en,1994,5.4
1,26581,76.105463,Sherman's March,Documentary,en,1985,6.9
2,105071,76.861355,Pig Hunt,Action Horror Thriller,en,2008,4.9
3,56561,77.737695,10 Questions for the Dalai Lama,Documentary,en,2006,7.4
4,6917,77.925724,Promised Land,Drama Romance,en,1987,5.1
5,5591,78.010890,Monkey Trouble,Comedy Family,en,1994,5.7
6,562,78.070797,Welcome to the Dollhouse,Comedy Drama,en,1996,7.2
7,6356,78.510366,Gunfight at the O.K. Corral,Western,en,1957,6.9
8,100505,78.516521,Monte Walsh,Action Western,en,2003,5.8
9,7625,78.591456,Girl,Comedy Drama Romance Musical,en,1998,4.6


In [69]:
rank(bcq_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,4186,9.822283,The Fortune Cookie,Romance Comedy,en,1966,7.4
1,4168,9.834513,Get Over It,Comedy Romance,en,2001,5.4
2,6917,9.870267,Promised Land,Drama Romance,en,1987,5.1
3,523,9.964449,Ruby in Paradise,Drama Romance,en,1993,6.2
4,360,9.976474,I Love Trouble,Action Comedy Romance,en,1994,5.4
5,2356,10.057261,Celebrity,Drama Comedy,en,1998,6.0
6,90576,10.131841,What's Your Number?,Comedy Romance,en,2011,6.4
7,78574,10.160169,Winter's Bone,Drama,en,2010,6.9
8,80789,10.198572,Loving Leah,Drama Romance TV Movie,en,2009,6.5
9,97705,10.206452,Where the Boys Are '84,Comedy Romance,en,1984,4.2


In [70]:
rank(bcq_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,4186,2.088166,The Fortune Cookie,Romance Comedy,en,1966,7.4
1,2082,2.148512,The Mighty Ducks,Comedy Family,en,1992,6.4
2,115348,2.166608,The Wise Kids,Drama,en,2011,6.2
3,7405,2.167757,Road to Bali,Comedy,en,1952,6.0
4,3076,2.176996,Irma la Douce,Comedy Romance,en,1963,7.7
5,53189,2.185992,Eagle vs Shark,Comedy Romance,en,2007,6.7
6,80789,2.196846,Loving Leah,Drama Romance TV Movie,en,2009,6.5
7,78,2.202449,The Crossing Guard,Drama Thriller,en,1995,6.0
8,96601,2.212819,Icon,TV Movie Action Drama Thriller,en,2005,6.2
9,66979,2.214460,Out At The Wedding,Romance,en,2007,5.7


In [57]:
rank(bcq_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,1271,0.688409,Fried Green Tomatoes,Drama Comedy,en,1991,7.8
1,71264,0.701323,Cloudy with a Chance of Meatballs,Animation Comedy Family,en,2009,6.5
2,123735,0.711345,Vito,Documentary,en,2011,8.2
3,49644,0.711665,Off the Black,Drama,en,2006,5.0
4,62733,0.715212,Quarantine,Horror Sci-Fi Thriller,en,2008,5.7
5,1747,0.717264,Wag the Dog,Comedy Drama,en,1997,6.9
6,82767,0.718790,Rabbit Hole,Drama,en,2010,6.8
7,34528,0.727032,Junebug,Comedy Drama,en,2005,6.7
8,95290,0.727822,Kind Lady,Drama Thriller,en,1935,7.2
9,59143,0.730848,Super High Me,Comedy Documentary,en,2007,5.6


In [71]:
rank(bcq_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,105071,84.160141,Pig Hunt,Action Horror Thriller,en,2008,4.9
1,6917,85.059219,Promised Land,Drama Romance,en,1987,5.1
2,360,85.072342,I Love Trouble,Action Comedy Romance,en,1994,5.4
3,4168,86.773560,Get Over It,Comedy Romance,en,2001,5.4
4,25915,88.476410,The Southerner,Drama,en,1945,5.8
5,523,88.501770,Ruby in Paradise,Drama Romance,en,1993,6.2
6,2356,88.960075,Celebrity,Drama Comedy,en,1998,6.0
7,6532,89.759888,Up the Sandbox,Drama Comedy,en,1972,6.9
8,4186,89.896904,The Fortune Cookie,Romance Comedy,en,1966,7.4
9,82911,90.248428,The Cowboy and the Lady,Comedy Romance,en,1938,6.5
